In [1]:
# Description: This script is used to create a workflow for the data analysis

import pandas as pd
import glob
import os
import sys

# Get the parent directory and add it to sys.path
#parent_dir = os.path.abspath("/notebooks/workflow_Eivind")
#sys.path.insert(0, parent_dir)

# Now import the module
from workflow_Eivind.ML_lib4 import *

# packages:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
#from tensorflow.keras import Sequential
#from tensorflow.keras.layers import Input, Dense, Dropout #, LSTM
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import xgboost as xgb
from xgboost import XGBRegressor
#from bartpy.sklearnmodel import SklearnModel as BART

print("Imported libraries\n")

# Script workflow:
# - load data
# - scaling
# - split into training and testing based on data gaps
# - machine learning models and applying hyperparameter search grids

#### to change: import from external argument ###
algorithm = 'linear_regression' #look below for the options
scoring_metrics = ['r2','mse','mae']

print(f"Machine Learning method: {algorithm}")
print(f"Evaluation metrics: {scoring_metrics}")

# save hyperparameters for the run as csv file?
save_to_csv = True

Imported libraries

Machine Learning method: linear_regression
Evaluation metrics: ['r2', 'mse', 'mae']


In [2]:
def scaling(dataframe, algorithm, cyclic_features=None, verbose=False, plot=False):

    scaling_method = None

    assert algorithm in ["linear_regression", "neural_network", "bart", "lstm", "random_forest", "xgboost"], \
        "Invalid algorithm specified."

    # Determine scaling method based on algorithm
    if algorithm in ["lstm", "neural_network"]:
        scaling_method = "minmax"
        if verbose: print(f"Algorithm '{algorithm}' selected: Using MinMax scaling.")
    elif algorithm == "linear_regression":
        scaling_method = "standard"
        if verbose: print(f"Algorithm '{algorithm}' selected: Using Standard scaling.")
    else:
        scaling_method = None
        if verbose: print(f"Algorithm '{algorithm}' selected: No scaling will be applied.")

    scaled_df = dataframe.copy()
    scaling_info = {}
    columns = [col for col in dataframe.columns]

    if verbose: print("Scaling in progress...\n")
    for col in columns:
        if verbose: print(f"Processing column: {col}")
        
        # Handle missing values
        if scaled_df[col].isnull().any():
            if verbose: print(f"Warning: Column '{col}' contains missing values. Imputing with median.")
            scaled_df[col].fillna(scaled_df[col].median(), inplace=True)

        # Cyclical encoding for specified features
        if cyclic_features and col in cyclic_features:
            max_value = scaled_df[col].max() + 1  # Assuming cyclic range [0, max_value - 1]
            if verbose: print(f"  Applying cyclical encoding (sine/cosine) for '{col}'.")
            scaled_df[f"{col}_sin"] = np.sin(2 * np.pi * scaled_df[col] / max_value)
            scaled_df[f"{col}_cos"] = np.cos(2 * np.pi * scaled_df[col] / max_value)
            scaling_info[col] = {"method": "cyclical_encoding"}
            scaled_df = scaled_df.drop(columns=col)
            continue

        # Skip scaling if no scaling method is selected
        if scaling_method is None:
            if verbose: print(f"No scaling applied for column: {col}")
            scaling_info[col] = {"method": "none"}
            continue

        # Determine the scaler
        if scaling_method == "minmax":
            scaler = MinMaxScaler()
            message = "Applying MinMaxScaler."
        elif scaling_method == "standard":
            scaler = StandardScaler()
            message = "Applying StandardScaler."
        elif scaling_method == "logminmax":
            scaled_df[col] = np.log1p(scaled_df[col] - scaled_df[col].min() + 1)
            scaler = MinMaxScaler()
            message = "Applying log transformation followed by MinMaxScaler."
        elif scaling_method == "individual":
            skewness = skew(scaled_df[col])
            _, p_value = normaltest(scaled_df[col])
            if verbose: print(f"  Skewness: {skewness:.2f}, Normality test p-value: {p_value:.4f}")
            if p_value > 0.05:  # Normally distributed
                scaler = StandardScaler()
                message = "Applying StandardScaler (data is approximately normal)."
            elif abs(skewness) > 1:  # Highly skewed
                scaled_df[col] = np.log1p(scaled_df[col] - scaled_df[col].min() + 1)
                scaler = MinMaxScaler()
                message = "Applying log transformation followed by MinMaxScaler (data is skewed)."
            else:  # Mildly skewed or uniform
                scaler = MinMaxScaler()
                message = "Applying MinMaxScaler (data is mildly skewed or uniform)."
        else:
            raise ValueError(f"Unknown scaling method: {scaling_method}")

        if verbose: print(message)
        
        # Apply scaling
        scaled_values = scaler.fit_transform(scaled_df[col].values.reshape(-1, 1))
        scaled_df[col] = scaled_values.flatten()
        scaling_info[col] = {"method": type(scaler).__name__}

        if scaling_method == "individual":
            scaling_info[col].update({
                "skewness": skewness,
                "normality_p_value": p_value,
            })

        if plot: plot_scaling(dataframe, scaled_df, col)

    return scaled_df, scaling_info


In [3]:
################################################################################
### Workflow Parameters
################################################################################

# Define features to run as input variables for the models:
features = [
    "SWin",
    "LWin",
    "Tair",
    "RH_air",
    "prec",
    "u",
    "snow_cover",
    "hour",
    "doy"
    ]

# Choose the gaps dataset - either structured or random gaps
gaps_data_file = 'random_gaps_1' # 'random_gaps_1' -- values from 1 to 5 for diff versions

# Define the cross-validation strategy:
CV_scoring = 'neg_mean_absolute_error'   # e.g. 'neg_mean_squared_error', 'r2', 'neg_root_mean_squared_error. More available methods for regression evaluation (scoring): https://scikit-learn.org/1.5/modules/model_evaluation.html#scoring-parameter)
cv = 3  # Number of cross-validation folds

################################################################################
### Data
################################################################################


##### Load the synthetic dataset:
# a. Load single CSV files in separate dfs
# b. Merge the dfs into one single "synthetic_dataset"

folder_path = '../data/synthetic_dataset' # NOTE: May need to adjust if the script is used from another folder

csv_files = glob.glob(os.path.join(folder_path, '*.csv')) # use glob library to find all CSV files

dfs = [] #to store individual DataFrames.

for file in csv_files:
    data = pd.read_csv(file, parse_dates=['time'], sep=',')
    # 'parse_dates' argument ensures the 'time' column is interpreted as datetime objects.
    
    dfs.append(data)

syn_ds = dfs[0] # Start with the first DataFrame as the base for merging.

for data in dfs[1:]:
    # Merge each subsequent DataFrame with the base DataFrame (`syn_ds`).
    # The merge is done using an ordered merge on the 'time' column.
    # This ensures that the merged dataset remains sorted by 'time'.
    syn_ds = pd.merge_ordered(syn_ds, data, on='time')

#-------------------------------------------------------------------------------
# Features and target variables:

syn_ds["time"] = pd.to_datetime(syn_ds["time"])
syn_ds["doy"] = syn_ds["time"].dt.dayofyear
syn_ds["hour"] = syn_ds["time"].dt.hour

y = syn_ds["LE"]
X = syn_ds[features]

#-------------------------------------------------------------------------------
# Split into training and testing datasets based on gaps in LE:

def split_train_test_dataset(original_X, original_y):
    # Function to load data gaps dataset
    def load_data_gaps(file_name):
        return pd.read_csv(f'../data/LE-gaps/{file_name}.csv', parse_dates=['time'], sep=',') # NOTE: May need to adjust if the script is used from another folder

    LE_gaps = load_data_gaps(gaps_data_file)

    # Select X and y where LE_gaps is not null
    X_train = original_X[LE_gaps['LE_gaps'].notnull()]
    y_train = original_y[LE_gaps['LE_gaps'].notnull()]

    # The following test set is for where there are data gaps
    X_test = original_X[LE_gaps['LE_gaps'].isnull()]

    # LE without gaps:
    LE = syn_ds['LE']
    # extract LE where LE_gaps is null:
    y_test = LE[LE_gaps['LE_gaps'].isnull()]

    print("Created training and testing datasets\n")
    return X_train, y_train, X_test, y_test

In [173]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluate_model(model, X_test, y_test, scoring):
    """
    Evaluate a trained model on the test data and compute specified metrics.

    Parameters:
        model (object): Trained machine learning model.
        X_test (array-like): Test feature matrix.
        y_test (array-like): Test target vector.
        scoring (list, optional): List of scoring metrics for evaluation (default is ['r2']).

    Returns:
        dict: A dictionary containing predictions and evaluation metrics.
    """
    # Validate the scoring parameter
    valid_metrics = {'r2', 'mse', 'mae'}
    invalid_metrics = [metric for metric in scoring if metric not in valid_metrics]
    if invalid_metrics:
        raise ValueError(f"Invalid scoring metric(s): {invalid_metrics}. Allowed values are {valid_metrics}.")
    
    try:
        # Make predictions on the test set
        y_pred = model.predict(X_test)
        
        # Compute evaluation metrics
        metrics = {}
        print(f"Test Metrics:")
        if 'r2' in scoring:
            r2 = r2_score(y_test, y_pred)
            metrics['r2'] = r2
            print(f"  R² Score: {r2:.2f}")
        if 'mse' in scoring:
            mse = mean_squared_error(y_test, y_pred)
            metrics['mse'] = mse
            print(f"  Mean Squared Error: {mse:.2f}")
        if 'mae' in scoring:
            mae = mean_absolute_error(y_test, y_pred)
            metrics['mae'] = mae
            print(f"  Mean Absolute Error: {mae:.2f}")
        
        return metrics
    
    except Exception as e:
        print(f"Error during model evaluation: {e}")
        return None


In [4]:
################################################################################
# MACHINE LEARNING MODELS
################################################################################

def machine_learning_method(algorithm):

    allowed_algorithms = {'linear_regression', 'random_forest', 'neural_network', 'lstm', 'xgboost', 'bart'}
    
    # Validate the algorithm
    assert algorithm in allowed_algorithms, (
        f"Invalid algorithm '{algorithm}'. "
        f"Allowed values are: {', '.join(allowed_algorithms)}."
    )
    
    if algorithm == 'linear_regression':

        # LINEAR REGRESSION (No hyperparameters to tune)
        
        # Apply scaling to input features
        X_scaled, _ = scaling(X, algorithm, cyclic_features = ['hour','doy'])
        
        # Split training and testing datasets
        X_train, y_train, X_test, y_test = split_train_test_dataset(X_scaled, y)
        
        # Apply machine learning method
        lr_model = LinearRegression()
        lr_model.fit(X_train, y_train)
        
        # Generate predictions
        y_pred = lr_model.predict(X_test)

        # Evaluate the ML method
        LR_metrics = evaluate_model(lr_model, X_test, y_test,scoring_metrics)
        metric = LR_metrics # for the general saving code at the end of the script

        print("\n=== LINEAR REGRESSION RESULTS ===")
        print(f"Test Metrics: {LR_metrics}")

        #-------------------------------------------------------------------------------
    elif algorithm == 'random_forest':

        # RANDOM FOREST REGRESSOR
        
        # Split training and testing datasets
        X_train, y_train, X_test, y_test = split_train_test_dataset(X, y)
        
        param_grid_rf = {
            'n_estimators': [100, 200, 300, 500],
            'max_features': ['sqrt', 'log2', None],
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4, 8]
        }

        rf_model = RandomForestRegressor()
        RF_best_model, RF_best_params, cv_results = model_tuning_CV(X_train, y_train, rf_model, param_grid_rf, cv, CV_scoring)
        RF_metrics = evaluate_model(RF_best_model, X_test, y_test, scoring_metrics)
        metric = RF_metrics # for the general saving code at the end of the script

        # Generate predictions
        y_pred = RF_best_model.predict(X_test)

        print("\n=== RANDOM FOREST RESULTS ===")
        print(f"Best Parameters: {RF_best_params}")
        print(f"Test Metrics: {RF_metrics}")

        #-------------------------------------------------------------------------------
    elif algorithm == 'neural_network':
    
        # NEURAL NETWORK REGRESSOR
        
        # Apply scaling to input features
        X_scaled, _ = scaling(X, algorithm, cyclic_features = ['hour','doy'])
        
        # Split training and testing datasets
        X_train, y_train, X_test, y_test = split_train_test_dataset(X_scaled, y)
        
        def create_NN_model(units=64, activation='relu'):
            model = Sequential([
                Input(shape=(X_train.shape[1],)),
                Dense(units, activation=activation, kernel_initializer='uniform'),
                Dropout(0.2),
                Dense(units, activation=activation),
                Dropout(0.2),
                Dense(1)
            ])
            model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
            return model

        keras_model = KerasRegressor(model=create_NN_model, verbose=0)

        param_grid_nn = {
            'model__units': [32, 64, 128],
            'model__activation': ['relu', 'tanh', 'sigmoid'],
            'batch_size': [10, 20, 30],
            'epochs': [50, 100, 200]
        }

        NN_best_model, NN_best_params, cv_results = model_tuning_CV(X_train, y_train, keras_model, param_grid_nn, cv, CV_scoring)
        NN_metrics = evaluate_model(NN_best_model, X_test, y_test, scoring_metrics)
        metric = NN_metrics # for the general saving code at the end of the script

        # Generate predictions
        y_pred = NN_best_model.predict(X_test)

        print("\n=== NEURAL NETWORK RESULTS ===")
        print(f"Best Parameters: {NN_best_params}")
        print(f"Test Metrics: {NN_metrics}")

        #-------------------------------------------------------------------------------
    elif algorithm == 'lstm':
    
        # LSTM REGRESSOR
        
        # Apply scaling to input features
        X_scaled, _ = scaling(X, algorithm, cyclic_features = ['hour','doy'])
        
        # Split training and testing datasets
        X_train, y_train, X_test, y_test = split_train_test_dataset(X_scaled, y)
        
        
        def create_lstm_model(units=64, activation='relu'):
            model = Sequential([
                Input(shape=(X_train.shape[1], 1)),
                LSTM(units, activation=activation, return_sequences=False),
                Dense(1)
            ])
            model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
            return model

        lstm_model = KerasRegressor(model=create_lstm_model, verbose=0)

        param_grid_lstm = {
            'model__units': [32, 64, 128],
            'model__activation': ['relu', 'tanh'],
            'batch_size': [10, 20],
            'epochs': [50, 100]
        }

        LSTM_best_model, LSTM_best_params, cv_results = model_tuning_CV(X_train, y_train, lstm_model, param_grid_lstm, cv, CV_scoring)
        LSTM_metrics = evaluate_model(LSTM_best_model, X_test, y_test, scoring_metrics)
        metric = LSTM_metrics # for the general saving code at the end of the script

        # Generate predictions
        y_pred = LSTM_best_model.predict(X_test)

        print("\n=== LSTM RESULTS ===")
        print(f"Best Parameters: {LSTM_best_params}")
        print(f"Test Metrics: {LSTM_metrics}")

    #-------------------------------------------------------------------------------
    elif algorithm == 'xgboost':
    
        # XGBOOST REGRESSOR
        
        # Apply scaling to input features
        X_scaled, _ = scaling(X, algorithm, cyclic_features = ['hour','doy'])
        
        # Split training and testing datasets
        X_train, y_train, X_test, y_test = split_train_test_dataset(X_scaled, y)

        param_grid_xgb = {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0]
        }

        xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
        XGB_best_model, XGB_best_params, cv_results = model_tuning_CV(X_train, y_train, xgb_model, param_grid_xgb, cv, CV_scoring)
        XGB_metrics = evaluate_model(XGB_best_model, X_test, y_test, scoring_metrics)
        metric = XGB_metrics # for the general saving code at the end of the script

        # Generate predictions
        y_pred = XGB_best_model.predict(X_test)

        print("\n=== XGBOOST RESULTS ===")
        print(f"Best Parameters: {XGB_best_params}")
        print(f"Test Metrics: {XGB_metrics}")

    #-------------------------------------------------------------------------------
    elif algorithm == 'bart':
    
        # BART REGRESSOR
        
        # Apply scaling to input features
        X_scaled, _ = scaling(X, algorithm, cyclic_features = ['hour','doy'])
        
        # Split training and testing datasets
        X_train, y_train, X_test, y_test = split_train_test_dataset(X_scaled, y)

        bart_model = BART(random_state=42)
        
        param_grid_bart = {
            'n_trees': [50, 100, 200],
            'alpha': [0.95, 0.99],
            'beta': [1.0, 2.0],
            'k': [2.0, 3.0]
        }

        BART_best_model, BART_best_params, cv_results = model_tuning_CV(X_train, y_train, bart_model, param_grid_bart, cv, CV_scoring)
        BART_metrics = evaluate_model(BART_best_model, X_test, y_test, scoring_metrics)
        metric = BART_metrics # for the general saving code at the end of the script

        # Generate predictions
        y_pred = bart_model.predict(X_test)

        print("\n=== BART RESULTS ===")
        print(f"Best Parameters: {BART_best_params}")
        print(f"Test Metrics: {BART_metrics}")
#-------------------------------------------------------------------------------
    # SAVING RESULTS
    
    # Save results to CSV

    if save_to_csv and algorithm != 'linear_regression':
    # Save CV results
        cv_results_df = pd.DataFrame(cv_results)
        cv_results_df.to_csv(f'../results/{algorithm}_{gaps_data_file}_cv_results.csv', index=False)

    if save_to_csv:
    # Save predictions
        results_df = pd.DataFrame({
            'index': X_test.index,
            'Actual': y_test,
            'Predicted': y_pred
        })
        results_df.to_csv(f'../results/{algorithm}_{gaps_data_file}_predictions.csv', index=False)

    # Save R² or MSE value if available
    if scoring_metrics != None:

        # Prepare the DataFrame with all metrics
        metric_df = pd.DataFrame([metric])  # Convert the dictionary into a DataFrame

        # Define the output file path
        metric_file_name = f'../results/{algorithm}_{gaps_data_file}_metrics_results.csv'

        # Save metrics to the CSV file
        metric_df.to_csv(metric_file_name, index=False)

        #if scoring_metrics == 'r2':
        #    metric_df = pd.DataFrame({'R2_Score': [metric]})
        #    metric_file_name = f'../results/{algorithm}_{gaps_data_file}_r2_results.csv'
        #elif scoring_metrics == 'mse':
        #    metric_df = pd.DataFrame({'MSE': [metric]})
        #    metric_file_name = f'../results/{algorithm}_{gaps_data_file}_mse_results.csv'

        #if scoring_metrics is not None:
        #    metric_df.to_csv(metric_file_name, index=False)

# Perform the function
machine_learning_method(algorithm)

Created training and testing datasets

Test Metrics:
  R² Score: 0.75
  Mean Squared Error: 957.09
  Mean Absolute Error: 22.64

=== LINEAR REGRESSION RESULTS ===
Test Metrics: {'r2': 0.7514328851756489, 'mse': np.float64(957.0945303337479), 'mae': np.float64(22.64125084110847)}
